In [13]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

In [14]:
data = pd.read_csv("incoming_packages.csv")
data.head(len(data))

,date,total_packages,cooled_packages,fire_packages,normal_packages
0,2020-01-01,1745,370,488,887
1,2020-01-02,1847,308,402,1137
2,2020-01-03,1842,314,394,1134
3,2020-01-04,664,131,181,352
4,2020-01-05,871,137,190,544
...,...,...,...,...,...
846,2022-04-26,2268,467,503,1298
847,2022-04-27,2466,432,641,1393
848,2022-04-28,2436,545,556,1335
849,2022-04-29,2302,368,547,1387


In [15]:
# data.plot()
# plt.tight_layout()
# plt.show()

# TODO: show dates instead of numbers on x-axis